<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/ResumeParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/SnorkelProject



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/SnorkelProject


In [7]:
!ls

Resume.docx  snorkel-tutorials


In [3]:
!pip install docx2txt
!pip install -U spacy

  Created wheel for docx2txt: filename=docx2txt-0.8-cp36-none-any.whl size=3963 sha256=d791fe7c9cac967a81edb5cd658814810f621bb61a7101b7ab54671898485876
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt
     |████████████████████████████████| 10.0MB 2.6MB/s 
     |████████████████████████████████| 2.1MB 45.9MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
import PyPDF2



fFileObj = open('Resume.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(fFileObj)
pageObj = pdfReader.getPage(0)
print("Total Pages : {} ".format(pdfReader.numPages))

resume = pageObj.extractText()

In [7]:
import docx2txt
import re
import spacy
from spacy.matcher import Matcher
import requests
import json



In [ ]:


def extract_text_from_doc(doc_path):
   temp = docx2txt.process(doc_path)
   #text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
   #return ' '.join(text)
   return temp


path = "sampleResume.docx"
text = extract_text_from_doc(path)

print(text)



In [38]:

pattern = '[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}'
match = re.findall(pattern, text)

print(match)

['emoore@email.com']


In [41]:
pattern = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'
match = re.findall(pattern, text)

print(match)

['(123) 456-7891']


In [13]:


# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
   nlp_text = nlp(resume_text)
  
   # First name and Last name are always Proper Nouns
   pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
  
   matcher.add('NAME', None, pattern)
  
   matches = matcher(nlp_text)
  
   for match_id, start, end in matches:
       span = nlp_text[start:end]
       return span.text



name = extract_name(text)
print(name)

Ebony Moore


In [ ]:
try:
    import spacy
    import json
except Exception as e:
    print(e)
    

class EntityGenerator(object):
    
    _slots__ = ['text']
    
    def __init__(self, text=None):
        self.text = text
        
    def get(self):
        """
        Return a Json
        """
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(self.text)
        text = [ent.text for ent in doc.ents]
        entity = [ent.label_ for ent in doc.ents]
    
        from collections import Counter
        import json

        data = Counter(zip(entity))
        unique_entity = list(data.keys())
        unique_entity = [x[0] for x in unique_entity]

        d = {}
        for val in unique_entity:
            d[val] = []

        for key,val in dict(zip(text, entity)).items():
            if val in unique_entity:
                d[val].append(key)
        return d

In [ ]:
helper = EntityGenerator(text=text)
response = helper.get()
print(json.dumps(response , indent=3))